In [2]:
import pandas as pd
import numpy as np


from my_modules import my_functions as mybib
from my_modules import project_functions as pr

In [3]:
raw_df = pd.read_pickle('../data/output/temp_analysis/accident_dataset.pkl')

raw_df.columns

Index(['key', 'object_id', 'old_lor', 'street_default', 'lor', 'year', 'month',
       'hour', 'weekday', 'ac_category', 'ac_type', 'ac_type2', 'ac_light',
       'is_bicycle', 'is_car', 'is_pedestrian', 'is_motorcycle', 'is_truck',
       'is_other', 'street_condition', 'linrefx', 'linrefy', 'xgcswgs84',
       'ygcswgs84', 'latitude', 'longitude', 'lor_3', 'lor_2', 'lor_1'],
      dtype='object')

Split the dataset into the model data (LOR information is available, used to generate train, test data) and the predict data (LOR information is missing, used to predict data)

In [4]:
df = raw_df[['key','lor', 'latitude', 'longitude']]

display(df.describe())

df[['lor']] = df[['lor']].astype(str)

df_model = df.loc[df['lor'] != '0']
df_predict = df.loc[df['lor'] == '0']

display(df_model)
display(df_predict)
X = df_model.drop(columns = ['lor','key'])
y = df_model['lor']


X_predict = df_predict.drop(columns = ['lor','key'])
display(X)
display(X_predict)



,latitude,longitude
count,50119.000000,50119.000000
mean,52.506569,13.389325
std,0.239221,0.198554
min,0.000090,13.092251
25%,52.477627,13.326074
50%,52.510125,13.387456
75%,52.538866,13.444886
max,52.660146,52.112612


C:\Users\Domen\AppData\Local\Temp\ipykernel_9564\3848328762.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['lor']] = df[['lor']].astype(str)


,key,lor,latitude,longitude
0,112695-2018-2500729,02500729,52.513597,13.475018
1,112705-2018-12500824,12500824,52.587259,13.291022
2,112726-2018-2400520,02400520,52.526019,13.420578
3,112737-2018-7200308,07200308,52.481844,13.348288
4,112747-2018-3200206,03200206,52.583472,13.403228
...,...,...,...,...
11262,112174-2021-4300414,04300414,52.511008,13.296345
11263,111541-2021-4400726,04400726,52.493813,13.289496
11264,110370-2021-2400520,02400520,52.527534,13.423095
11265,109998-2021-7300619,07300619,52.474367,13.359259


,key,lor,latitude,longitude
4501,199484-2018-0,0,52.546779,13.603738
5026,200009-2018-0,0,52.391767,13.130701
5339,200322-2018-0,0,52.549535,13.587510
0,49090-2019-0,0,52.586085,13.341464
1,49091-2019-0,0,52.558615,13.413563
...,...,...,...,...
13387,202777-2019-0,0,52.515438,13.418082
13388,202778-2019-0,0,52.515687,13.498618
13389,202779-2019-0,0,52.516401,13.380820
2715,136842-2020-0,0,52.417671,13.449569


,latitude,longitude
0,52.513597,13.475018
1,52.587259,13.291022
2,52.526019,13.420578
3,52.481844,13.348288
4,52.583472,13.403228
...,...,...
11262,52.511008,13.296345
11263,52.493813,13.289496
11264,52.527534,13.423095
11265,52.474367,13.359259


,latitude,longitude
4501,52.546779,13.603738
5026,52.391767,13.130701
5339,52.549535,13.587510
0,52.586085,13.341464
1,52.558615,13.413563
...,...,...
13387,52.515438,13.418082
13388,52.515687,13.498618
13389,52.516401,13.380820
2715,52.417671,13.449569


In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)

In [6]:
from sklearn.neighbors import KNeighborsClassifier

KNN = KNeighborsClassifier(n_neighbors=2, p=1, weights = 'distance')
KNN.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=2, p=1, weights='distance')

Test score of 0.95 -> will use the model to predict the missing values

In [9]:
score = KNN.score(X_test, y_test)
print(f'Test Score: {score}')


Test Score: 0.9503063308373043


In [8]:
# y_predicted

NameError: name 'y_predicted' is not defined

In [10]:
y_predicted = KNN.predict(X_predict)
df_predict['lor'] = y_predicted
df_predict.drop(columns = ['latitude', 'longitude'])

C:\Users\Domen\AppData\Local\Temp\ipykernel_9564\953877691.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_predict['lor'] = y_predicted


,key,lor
4501,199484-2018-0,10200422
5026,200009-2018-0,06400735
5339,200322-2018-0,10200417
0,49090-2019-0,12500930
1,49091-2019-0,03400826
...,...,...
13387,202777-2019-0,01100311
13388,202778-2019-0,11300725
13389,202779-2019-0,01100206
2715,136842-2020-0,08200833


,key,lor
4501,199484-2018-0,10200422
5026,200009-2018-0,06400735
5339,200322-2018-0,10200417
0,49090-2019-0,12500930
1,49091-2019-0,03400826
...,...,...
13387,202777-2019-0,01100311
13388,202778-2019-0,11300725
13389,202779-2019-0,01100206
2715,136842-2020-0,08200833


In [ ]:
df

,key,lor,latitude,longitude
0,112695-2018-2500729,02500729,52.513597,13.475018
1,112705-2018-12500824,12500824,52.587259,13.291022
2,112726-2018-2400520,02400520,52.526019,13.420578
3,112737-2018-7200308,07200308,52.481844,13.348288
4,112747-2018-3200206,03200206,52.583472,13.403228
...,...,...,...,...
11262,112174-2021-4300414,04300414,52.511008,13.296345
11263,111541-2021-4400726,04400726,52.493813,13.289496
11264,110370-2021-2400520,02400520,52.527534,13.423095
11265,109998-2021-7300619,07300619,52.474367,13.359259


Merge the predicted values into the old total dataset

In [ ]:
raw_df = pd.merge(raw_df, df_predict, on = 'key', how = 'left')
raw_df['lor_x'] = raw_df.apply(lambda row: row['lor_y'] if row['lor_x'] == '0' else row['lor_x'], axis=1)
raw_df.drop(columns = ['latitude_y', 'longitude_y', 'lor_y'], inplace = True)

In [ ]:
raw_df.columns

,key,object_id,old_lor,street_default,lor_x,year,month,hour,weekday,ac_category,...,xgcswgs84,ygcswgs84,latitude_x,longitude_x,lor_3,lor_2,lor_1,lor,latitude,longitude
0,112695-2018-2500729,112695,02050602,Samariterviertel,02500729,2018,1,15,4,3,...,13.475018,52.513597,52.513597,13.475018,025007,0250,02,NaN,NaN,NaN
1,112705-2018-12500824,112705,12304314,Ziekowstraße/Freie Scholle,12500824,2018,1,11,2,3,...,13.291022,52.587259,52.587259,13.291022,125008,1250,12,NaN,NaN,NaN
2,112726-2018-2400520,112726,02040501,Barnimkiez,02400520,2018,1,9,3,3,...,13.420578,52.526019,52.526019,13.420578,024005,0240,02,NaN,NaN,NaN
3,112737-2018-7200308,112737,07020202,Volkspark (Rudolf-Wilde-Park),07200308,2018,1,17,2,3,...,13.348288,52.481844,52.481844,13.348288,072003,0720,07,NaN,NaN,NaN
4,112747-2018-3200206,112747,03020209,Niederschönhausen,03200206,2018,1,15,4,3,...,13.403228,52.583472,52.583472,13.403228,032002,0320,03,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50114,112174-2021-4300414,112174,0,0,04300414,2021,3,16,4,3,...,13.296345,52.511008,52.511008,13.296345,043004,0430,04,NaN,NaN,NaN
50115,111541-2021-4400726,111541,0,0,04400726,2021,1,1,6,3,...,13.289496,52.493813,52.493813,13.289496,044007,0440,04,NaN,NaN,NaN
50116,110370-2021-2400520,110370,0,0,02400520,2021,1,20,1,3,...,13.423095,52.527534,52.527534,13.423095,024005,0240,02,NaN,NaN,NaN
50117,109998-2021-7300619,109998,0,0,07300619,2021,1,14,7,3,...,13.359259,52.474367,52.474367,13.359259,073006,0730,07,NaN,NaN,NaN
